In [ ]:
# pip install spotipy

In [ ]:
# import packages
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import sys
import time 

In [ ]:
# use the spotipy package to establish a connection with the api dashboard 
client_id = '...'
client_secret = '...' # this might change with every session

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
# playlist retreival with help from: https://github.com/nathanwilthomas/Spotify_KNN/blob/master/Spotify%20KNN.ipynb
# Retrieves all songs from an identified playlist, and takes the ID, song, album, artists and puts this all into a DataFrame


def analyze_playlist(url):
   # song name
    offset = 0
    name = []
    while True:
        response = sp.playlist_tracks(url,
                                      offset = offset,
                                      fields = ['items.track.name, total'])
        name.append(response["items"])
        offset = offset + len(response['items']) 
        if len(response['items']) == 0:
            break
    name_list = [b["track"]["name"] for a in name for b in a]    


    # album name
    offset = 0
    album = []
    while True:
        response = sp.playlist_tracks(url,
                                      offset = offset,
                                      fields = ['items.track.album.name,total'])
        album.append(response["items"])
        offset = offset + len(response['items'])
        if len(response['items']) == 0:
            break
    album_list = [b["track"]["album"]["name"] for a in album for b in a] 


   # artist name
    offset = 0
    artist = []
    while True:
        response = sp.playlist_tracks(url,
                                      offset = offset,
                                      fields = ['items.track.album.artists.name,total'])
        artist.append(response["items"])
        offset = offset + len(response['items'])       
        if len(response['items']) == 0:
            break
    artist_list = [b["track"]["album"]["artists"][0]["name"] for a in artist for b in a]

    
    # spotify id    
    offset = 0
    identifier = []
    while True:
        response = sp.playlist_tracks(url,
                                      offset = offset,
                                      fields = ['items.track.id,total'])
        identifier.append(response["items"])
        offset = offset + len(response['items'])      
        if len(response['items']) == 0:
            break
    identifier_list = [b["track"]["id"] for a in identifier for b in a]

    # popularity 
    offset = 0
    popularity = []
    while True:
        response = sp.playlist_tracks(url,
                                      offset = offset,
                                      fields = ['items.track.popularity,total'])
        popularity.append(response["items"])
        offset = offset + len(response['items'])      
        if len(response['items']) == 0:
            break
    popularity_list = [b["track"]["popularity"] for a in popularity for b in a]
    
    # Get audio features
    features = [sp.audio_features(identifier) for identifier in identifier_list]

    # Get each invidividual feature
    danceability = [(b["danceability"]) for a in features for b in a]    
    mode = [(b["mode"]) for a in features for b in a]
    energy = [(b["energy"]) for a in features for b in a]
    key = [(b["key"]) for a in features for b in a]        
    loudness = [(b["loudness"]) for a in features for b in a]       
    speechiness = [(b["speechiness"]) for a in features for b in a]
    acousticness = [(b["acousticness"]) for a in features for b in a]        
    instrumentalness = [(b["instrumentalness"]) for a in features for b in a] 
    liveness = [(b["liveness"]) for a in features for b in a]
    valence = [(b["valence"]) for a in features for b in a]        
    tempo = [(b["tempo"]) for a in features for b in a] 
    duration_ms = [(b["duration_ms"]) for a in features for b in a] 
    identifier_ = [(b["id"]) for a in features for b in a] 
        
    # dataframe (saved with current time)

    # for testing...

    # print(len(key))
    # print(len(danceability))
    # print(len(popularity_list))

    df = pd.DataFrame({"popularity": popularity_list,
                       "name": name_list, 
                       "artist": artist_list, 
                       "album": album_list, 
                       "id": identifier_list})
    
    df_2 = pd.DataFrame({"dance":danceability,
                         "mode":mode,
                         "energy":energy,
                         "key":key,
                         "loudness":loudness,
                         "speech":speechiness,
                         "acous":acousticness,
                         "instrum":instrumentalness,
                         "live":liveness,
                         "valence":valence,
                         "tempo":tempo,
                         "duration_(ms)": duration_ms,
                         "id_check":identifier_
                          })

    df_combined = df_2.join(df)
    
    df_combined.to_csv("df" + ".csv")

    return df_combined.tail()

In [ ]:
# the cleaning/analysis for this project is done in R 

analyze_playlist("2N8sqpn96q5oYiQepY7inT")

# rolling stone: 5dxn0i8MPl6XFVVxNatd6U"
# test set: 7KCecUfCC2fWgyeuTDDY0S
# out of sample high "valence" set: "2N8sqpn96q5oYiQepY7inT"